# SDialog dependencies

In [1]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name jsalt python=3.9 -y
!conda activate jsalt
!conda install sox
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt

In [2]:
import os
import json
from tqdm import tqdm

In [3]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

/Users/yanislabrak/opt/miniconda3/envs/jsalt-Py3-10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Persona

In [4]:
sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

In [5]:
%%script false --no-raise-error
persona_card_folder = "./personas"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [6]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [7]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [8]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [9]:
if FORCE_DIALOG_GENERATION:

    original_dialog = agent1.talk_with(agent2, max_turns=3)
    original_dialog.to_file("dialog_demo.json")

else:
    original_dialog = Dialog.from_file("dialog_demo.json")

original_dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio Generation

You can generate three type of audios:
- (default) Step 1: Raw utterances passed to a TTS model and concatenated to each others to create an audio file
- Step 2: Audio generated from multiple channels create using signal positions
- Step 3: Audio generated using room spacialization and multi-channels positions

If you want to trigger the "step 2" you need to give a Scaper argument to the `audio_pipeline`. While for the "step 3" you need also to give a "Room" in the `inference` function.

### Instanciate voices database

In [10]:
%%script false --no-raise-error
from sdialog.audio.voice_database import DummyVoiceDatabase
dummy_voice_database = DummyVoiceDatabase()

OR

In [11]:
%%script false --no-raise-error
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")

OR

In [12]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
jsalt_voices = HuggingfaceVoiceDatabase("sdialog/voices-jsalt")

### Instanciate TTS model

In [ ]:
%%script false --no-raise-error
from sdialog.audio.tts_engine import KokoroTTS
tts_pipeline = KokoroTTS()

OR

In [14]:
#  conda activate jsalt-Py3-10
import torchaudio
from sdialog.audio.tts_engine import IndexTTS
tts = IndexTTS()
audio, sampling_rate = tts.generate(
    "Brno is the best city in the planet, you know? and Loco Polaco is the craziest person I know",
    "./sergio.wav"
)
soundfile.write("./test_index_tts.wav", audio, sampling_rate)

>> GPT weights restored from: model/gpt.pth
Removing weight norm...
>> bigvgan weights restored from: model/bigvgan_generator.pth
>> TextNormalizer loaded
>> bpe model loaded from: model/bpe.model
>> start inference...


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


>> Reference audio length: 13.89 seconds
>> gpt_gen_time: 36.41 seconds
>> gpt_forward_time: 2.87 seconds
>> bigvgan_time: 9.29 seconds
>> Total inference time: 48.72 seconds
>> Generated audio length: 6.27 seconds
>> RTF: 7.7674


NameError: name 'soundfile' is not defined

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

## Step 1 : Concatenated utterances

In [ ]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
)

OR

In [ ]:
%%script false --no-raise-error
audio_pipeline = AudioPipeline() # Default values are used

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(dialog) # Generate the audio for the dialog
print(dialog.audio_step_1_filepath) # Path to the audio of the first stage of the audio pipeline

## Step 2: dScaper

In [ ]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
%%script false --no-raise-error
%pip install -r ../../../requirements-dscaper.txt

In [ ]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
audio_pipeline = AudioPipeline(dscaper=dsc)

In [ ]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(dialog)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)

## Step 3 : Room Accoustics

In [ ]:
audio_pipeline = AudioPipeline(dscaper=dsc) # The audio pipeline doesn't change

In [ ]:
from sdialog.audio.room import MicrophonePosition
from sdialog.audio.room_generator import RoomGenerator, RoomRole

In [ ]:
room = RoomGenerator().generate(RoomRole.CONSULTATION)
print(room)

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.CEILING_CENTERED
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepath)

## Automatic Position and SNR completion (under construction)

In [ ]:
from sdialog.audio.audio_events_enricher import AudioEventsEnricher
enricher = AudioEventsEnricher()

In [ ]:
audio_pipeline = AudioPipeline(enricher=enricher)

In [ ]:
for turn in dialog.turns:
    print(turn.text)
    print(turn.position)
    print(turn.snr)
    print("____________________")

## dialog.to_audio()

In [ ]:
# TODO: Add a demo of the audio generation pipeline from the dialog object

# Audio Evaluation

## Utterance level evaluation

In [ ]:
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

In [ ]:
# Utterances level evaluation
metrics_utterances_level = compute_evaluation_utterances(dialog)
for key, value in metrics_utterances_level.items():
    print(f"{key}: {value}")

# Audio level evaluation
metrics_audio_level = compute_evaluation_audio(dialog)
for key, value in metrics_audio_level.items():
    print(f"{key}: {value}")

## Audio level evaluation

In [ ]:
# TODO: Evaluate the final generated audio